In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Indicators.csv')
data.shape

(5656458, 6)

In [3]:
# 여기서부터 관련도 높은 feature 뽑는 것
target_id = 'Adolescent fertility rate (births per 1,000 women ages 15-19)';

target = data[data['IndicatorName']==target_id]

In [4]:
rel_columns_8 = ['Adolescent fertility rate (births per 1,000 women ages 15-19)',
 'Birth rate, crude (per 1,000 people)',
 'Net enrolment rate, secondary, both sexes (%)',
 'Net enrolment rate, secondary, female (%)',
 'Net enrolment rate, secondary, male (%)',
 'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)',
 'Mortality rate, under-5, female (per 1,000 live births)',
 'Mortality rate, under-5, male (per 1,000 live births)',
 'Prevalence of anemia among children (% of children under 5)',
 'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)',
 'Cause of death, by non-communicable diseases (% of total)',
 'Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)',
 'Survey mean consumption or income per capita, total population (2005 PPP $ per day)']

In [5]:
rel_columns_7 = ['Adolescent fertility rate (births per 1,000 women ages 15-19)',
 'Age dependency ratio (% of working-age population)',
 'Age dependency ratio, young (% of working-age population)',
 'Birth rate, crude (per 1,000 people)',
 'Fertility rate, total (births per woman)',
 'Life expectancy at birth, female (years)',
 'Life expectancy at birth, male (years)',
 'Life expectancy at birth, total (years)',
 'Mortality rate, adult, female (per 1,000 female adults)',
 'Mortality rate, infant (per 1,000 live births)',
 'Mortality rate, under-5 (per 1,000)',
 'Population, ages 0-14 (% of total)',
 'Population, ages 15-64 (% of total)',
 'Survival to age 65, female (% of cohort)',
 'Survival to age 65, male (% of cohort)',
 'Adjusted net enrolment rate, primary, both sexes (%)',
 'Adjusted net enrolment rate, primary, female (%)',
 'Gross enrolment ratio, secondary, both sexes (%)',
 'Gross enrolment ratio, secondary, female (%)',
 'Gross enrolment ratio, secondary, male (%)',
 'Lower secondary completion rate, both sexes (%)',
 'Net enrolment rate, secondary, both sexes (%)',
 'Net enrolment rate, secondary, female (%)',
 'Net enrolment rate, secondary, male (%)',
 'Primary completion rate, both sexes (%)',
 'Primary completion rate, female (%)',
 'Survival rate to the last grade of primary education, both sexes (%)',
 'Lower secondary completion rate, female (%)',
 'Lower secondary completion rate, male (%)',
 'Survival rate to the last grade of primary education, female (%)',
 'Survival rate to the last grade of primary education, male (%)',
 'Survival rate to Grade 5 of primary education, female (%)',
 'Youth literacy rate, population 15-24 years, both sexes (%)',
 'Youth literacy rate, population 15-24 years, male (%)',
 'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)',
 'Wanted fertility rate (births per woman)',
 'Access to electricity (% of population)',
 'Access to electricity, rural (% of rural population)',
 'Improved sanitation facilities (% of population with access)',
 'Improved sanitation facilities, rural (% of rural population with access)',
 'Improved sanitation facilities, urban (% of urban population with access)',
 'Improved water source (% of population with access)',
 'Improved water source, rural (% of rural population with access)',
 'Lifetime risk of maternal death (%)',
 'Maternal mortality ratio (modeled estimate, per 100,000 live births)',
 'Mortality rate, infant, female (per 1,000 live births)',
 'Mortality rate, infant, male (per 1,000 live births)',
 'Mortality rate, neonatal (per 1,000 live births)',
 'Mortality rate, under-5, female (per 1,000 live births)',
 'Mortality rate, under-5, male (per 1,000 live births)',
 'Prevalence of anemia among children (% of children under 5)',
 'Prevalence of anemia among pregnant women (%)',
 'Renewable energy consumption (% of total final energy consumption)',
 'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)',
 'Cause of death, by non-communicable diseases (% of total)',
 'Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)',
 'Survey mean consumption or income per capita, total population (2005 PPP $ per day)']

In [6]:
# 해당하는 indicator해서 그 데이터 프레임 사용하기 편하게 바꿔주는거 
def transformDf(data, ind):
    data2000=data[(data['Year'] >= 2000) & (data['Year'] < 2015) ]
    col8 = data2000[(data2000['IndicatorName'].isin(ind))]
    col8=col8.reset_index(drop=True)
    name_list = list()

    for i in col8.index:
        name_list.append( col8.loc[i,'CountryName'] + "_" + str(col8.loc[i,'Year']))
    col8["name"] = name_list
    
    col8=col8.pivot_table(values="Value",index="name",columns="IndicatorName")

    col8=col8.dropna(subset=[target_id])
    return col8


In [7]:
df=transformDf(data,rel_columns_8)

In [8]:

def make_grade(merged, k):
    min_value = min(merged[target_id])
    max_value = max(merged[target_id])
    value=(max_value-min_value)/k
    for i in range(k):
        for j in merged.index:
            if merged.loc[j, target_id] >=min_value + value*i and merged.loc[j, target_id] <= min_value + value*(i+1):
                merged.loc[j, 'grade'] = i
    return merged


In [9]:
def preprocessingMissing(df):
    
    #전체 개수랑 null 값이 비슷 하게 나오는 경우에는 그냥 아예 그 컬럼을 안쓰는게 나을 거 가음
    for i in df.columns:
        if df.isna().sum()[i]/len(df) >=0.7:
            df=df.drop(columns=[i])
    return df
        

In [10]:
df.head()

IndicatorName,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Birth rate, crude (per 1,000 people)","Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)","Cause of death, by non-communicable diseases (% of total)","Mortality rate, under-5, female (per 1,000 live births)","Mortality rate, under-5, male (per 1,000 live births)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, secondary, female (%)","Net enrolment rate, secondary, male (%)",Prevalence of anemia among children (% of children under 5),"Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)","Survey mean consumption or income per capita, total population (2005 PPP $ per day)",Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)
name,,,,,,,,,,,,,
Afghanistan_2000,153.8456,48.332,60.5,26.4,132.7,141.1,NaN,NaN,NaN,46.8,NaN,NaN,NaN
Afghanistan_2001,150.0468,47.835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.4,NaN,NaN,NaN
Afghanistan_2002,146.2480,47.230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.2,NaN,NaN,NaN
Afghanistan_2003,140.4764,46.538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.3,NaN,NaN,NaN
Afghanistan_2004,134.7048,45.761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.4,NaN,NaN,NaN


In [11]:
df = make_grade(df, 3)
df.head()

IndicatorName,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Birth rate, crude (per 1,000 people)","Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)","Cause of death, by non-communicable diseases (% of total)","Mortality rate, under-5, female (per 1,000 live births)","Mortality rate, under-5, male (per 1,000 live births)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, secondary, female (%)","Net enrolment rate, secondary, male (%)",Prevalence of anemia among children (% of children under 5),"Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)","Survey mean consumption or income per capita, total population (2005 PPP $ per day)",Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant),grade
name,,,,,,,,,,,,,,
Afghanistan_2000,153.8456,48.332,60.5,26.4,132.7,141.1,NaN,NaN,NaN,46.8,NaN,NaN,NaN,2.0
Afghanistan_2001,150.0468,47.835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.4,NaN,NaN,NaN,2.0
Afghanistan_2002,146.2480,47.230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.2,NaN,NaN,NaN,2.0
Afghanistan_2003,140.4764,46.538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.3,NaN,NaN,NaN,1.0
Afghanistan_2004,134.7048,45.761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.4,NaN,NaN,NaN,1.0


In [29]:
import numpy as np

# Exponent function for estimating e
exponent_func = np.vectorize(lambda x, std: np.exp((-1.0*x)/(2 * std)))

# Performs the E step give a list of x values, a list of means and a matrix of estimated values 
def E_step(x_list, mean_list, e_matrix):
    # Compute the standard deviation
    std_list = get_std(x_list, mean_list, e_matrix)
    estimated = list()
    for i,std in enumerate(std_list):
        estimated.append(exponent_func(np.square(x_list - mean_list[i]), std))
    # Calculating denominator
    estimated = np.array(estimated).transpose()
    for i, n in enumerate(estimated):
        estimated[i] /= n.sum()
    return estimated

# Performs the M step give a list of x values, a list of means
def M_step(x_list, e_matrix):
    # Calculating numerator
    numerator = np.dot(x_list, e_matrix)
    # Calculating denominator
    denominator = e_matrix.sum(axis=0)
    return np.divide(numerator, denominator)

# Compute the standard deviation
def get_std(x_list, mean_list, e_matrix):
    x_vector = x_list[np.newaxis].transpose()
    var = np.square(mean_list - x_vector) * e_matrix
    return np.sqrt(var.sum(axis=0)/e_matrix.sum(axis=0))

# Computes theta
def get_theta(e_matrix):
    return e_matrix.sum(axis=0) / len(e_matrix)

# Performs E-M for a number of steps
def simulate_E_M(x_list, e_matrix ,steps):
    mean_matrix = list()
    for i in range(steps):
        mean_list = M_step(x_list, e_matrix)
        mean_matrix.append(mean_list)
        e_matrix  = E_step(x_list, mean_list, e_matrix)
    return np.array(mean_matrix).transpose()


In [75]:
input_list = np.array(df)
df=df.fillna(0)
df.head()

IndicatorName,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Birth rate, crude (per 1,000 people)","Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)","Cause of death, by non-communicable diseases (% of total)","Mortality rate, under-5, female (per 1,000 live births)","Mortality rate, under-5, male (per 1,000 live births)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, secondary, female (%)","Net enrolment rate, secondary, male (%)",Prevalence of anemia among children (% of children under 5),"Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)","Survey mean consumption or income per capita, total population (2005 PPP $ per day)",Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant),grade
name,,,,,,,,,,,,,,
Afghanistan_2000,153.8456,48.332,60.5,26.4,132.7,141.1,0.0,0.0,0.0,46.8,0.0,0.0,0.0,2.0
Afghanistan_2001,150.0468,47.835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.4,0.0,0.0,0.0,2.0
Afghanistan_2002,146.2480,47.230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.2,0.0,0.0,0.0,2.0
Afghanistan_2003,140.4764,46.538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.3,0.0,0.0,0.0,1.0
Afghanistan_2004,134.7048,45.761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.4,0.0,0.0,0.0,1.0


In [76]:
x_list = input_list[:,0]
e_matrix = input_list[:,1:]
mean_matrix = simulate_E_M(x_list, e_matrix, 100)
print(mean_matrix)

[[ 73.3425347   69.07821466  65.19930695 ...  59.79037095  59.78888378
   59.7874367 ]
 [ 90.4496448   88.91915605  85.42830893 ...  81.28287179  81.27502697
   81.26768903]
 [ 42.61549694  30.81355731  34.5892706  ...  29.35052664  29.35051367
   29.35050086]
 ...
 [ 46.91631541  49.66724333  49.65268462 ...  40.17019612  40.1701435
   40.1700916 ]
 [109.63345029 129.09326451 133.70926921 ... 176.70737745 176.70262677
  176.69821562]
 [119.03988543 137.58821917 143.78240754 ... 131.51493073 131.50944044
  131.50442242]]
